# The [Import Loader](https://docs.python.org/3/reference/import.html#loaders)

`importnb` uses context manager to import Notebooks as Python packages and modules.  `importnb.Notebook` simplest context manager.  It will find and load any notebook as a module.

    >>> m = Notebook().from_filename('loader.ipynb', 'importnb.notebooks')
    >>> assert m and m.Notebook
 
     
### `importnb.Partial` 

    >>> with Partial(): 
    ...     from importnb.notebooks import loader
    >>> assert loader.__exception__ is None
    
## There is a [lazy importer]()

The Lazy importer will delay the module execution until it is used the first time.  It is a useful approach for delaying visualization or data loading.

    >>> with Lazy(): 
    ...     from importnb.notebooks import loader
    
## Loading from resources

Not all notebooks may be loaded as modules throught the standard python import finder.  `from_resource`, or the uncomfortably named `Notebook.from_filename` attributes, support [`importlib_resources`]() style imports and raw file imports.

    >>> from importnb.loader import from_resource
    >>> assert from_resource(Notebook(), 'loader.ipynb', 'importnb.notebooks')
    >>> assert Notebook().from_filename('loader.ipynb', 'importnb.notebooks')
    >>> assert Notebook().from_filename(m.__file__)
    

## Capturing stdin, stdout, and display objects

    >>> with Notebook(stdout=True, stderr=True, display=True, globals=dict(show=True)):
    ...     from importnb.notebooks import loader
    >>> assert loader.__output__
    
    # loader.__output__.stdout

In [1]:
    if 'show' in globals(): print("Catch me if you can")

In [2]:
    try:
        from .capture import capture_output
        from .decoder import identity, loads, dedent, cell_to_ast
    except:
        from capture import capture_output
        from decoder import identity, loads, dedent, cell_to_ast

    import inspect, sys, ast
    from copy import copy
    from importlib.machinery import SourceFileLoader
    from importlib.util import spec_from_loader

    from importlib._bootstrap import _call_with_frames_removed, _new_module
    try:  
        from importlib._bootstrap import _init_module_attrs, _call_with_frames_removed
        from importlib._bootstrap_external import FileFinder
        from importlib.util import module_from_spec
    except:
        #python 3.4
        from importlib.machinery import FileFinder
        from importlib._bootstrap import _SpecMethods

        def module_from_spec(spec): 
            return _SpecMethods(spec).create()
        
        def _init_module_attrs(spec, module):
            return _SpecMethods(spec).init_module_attrs(module)
    
    from io import StringIO
    from functools import partialmethod, partial
    from importlib import reload
    from traceback import print_exc, format_exc
    from warnings import warn
    from contextlib import contextmanager, ExitStack
    from pathlib import Path
    try:
        from importlib.resources import path
    except:
        from importlib_resources import path
    
    __all__ = 'Notebook', 'Partial', 'reload', 'Lazy'

## `sys.path_hook` modifiers

In [3]:
    @contextmanager
    def modify_file_finder_details():
        """yield the FileFinder in the sys.path_hooks that loads Python files and assure
        the import cache is cleared afterwards.  
        
        Everything goes to shit if the import cache is not cleared."""
        
        for id, hook in enumerate(sys.path_hooks):
            try:
                closure = inspect.getclosurevars(hook).nonlocals
            except TypeError: continue
            if issubclass(closure['cls'], FileFinder):
                sys.path_hooks.pop(id)
                details = list(closure['loader_details'])
                yield details
                break
        sys.path_hooks.insert(id, FileFinder.path_hook(*details))
        sys.path_importer_cache.clear()

Update the file_finder details with functions to append and remove the [loader details](https://docs.python.org/3.7/library/importlib.html#importlib.machinery.FileFinder).

In [4]:
    def add_path_hooks(loader: SourceFileLoader, extensions, *, position=0):
        """Update the FileFinder loader in sys.path_hooks to accomodate a {loader} with the {extensions}"""
        with modify_file_finder_details() as details:
            details.insert(position, (loader, extensions))

    def remove_one_path_hook(loader):
        loader = lazy_loader_cls(loader)
        with modify_file_finder_details() as details:
            _details = list(details)
            for ct, (cls, ext) in enumerate(_details):
                cls = lazy_loader_cls(cls)
                if cls == loader:
                    details.pop(ct)
                    break

In [5]:
    def lazy_loader_cls(loader):
        """Extract the loader contents of a lazy loader in the import path."""
        try:
            return inspect.getclosurevars(loader).nonlocals.get('cls', loader)
        except:
            return loader

## Loader Context Manager

`importnb` uses a context manager to assure that the traditional import system behaviors as expected.  If the loader is permenantly available then it may create some unexpected import behaviors.

In [6]:
    class ImportNbException(BaseException):
        """ImportNbException allows all exceptions to be raised, a null except statement always passes."""

In [7]:
    class PathHooksContext:
        def __enter__(self, position=0):  
            add_path_hooks(self.prepare(self), self.EXTENSION_SUFFIXES, position=position)
            return self
        
        def __exit__(self, *excepts): remove_one_path_hook(self)

        def prepare(self, loader):
            if self._lazy: 
                try:
                    from importlib.util import LazyLoader
                    if self._lazy: 
                        loader = LazyLoader.factory(loader)
                except:
                    ImportWarning("""LazyLoading is only available in > Python 3.5""")
            return loader


In [8]:
    def from_resource(loader, file=None, resource=None, exec=True, **globals):
        """Load a python module or notebook from a file location.

        from_filename is not reloadable because it is not in the sys.modules.

        This still needs some work for packages.
        
        >>> assert from_resource(Notebook(), 'loader.ipynb', 'importnb.notebooks')
        """
        with ExitStack() as stack:
            if resource is not None:
                file = Path(stack.enter_context(path(resource, file)))
            else:
                file = Path(file or loader.path)            
            name = (getattr(loader, 'name', False) == '__main__' and '__main__') or file.stem
            if file.suffixes[-1] == '.ipynb':  
                loader = loader(name, file)
            else:  
                loader = SourceFileLoader(name, str(file))

            lazy = getattr(loader, '_lazy', False)
            if lazy: 
                try:
                    from importlib.util import LazyLoader
                    loader = LazyLoader(loader)
                except:
                    ImportWarning("""LazyLoading is only available in > Python 3.5""")

            module = module_from_spec(spec_from_loader(name, loader))
            if exec:
                stack.enter_context(modify_sys_path(file))
                module.__loader__.exec_module(module, **globals)
        return module

In [9]:
    @contextmanager
    def modify_sys_path(file):
        """This is only invoked when using from_resource."""
        path = str(Path(file).parent)
        if path not in map(str, map(Path, sys.path)):
            yield sys.path.insert(0, path)
            sys.path = [object for object in sys.path if str(Path(object)) != path]
        else: yield

In [10]:
    class Notebook(SourceFileLoader, PathHooksContext, capture_output, ast.NodeTransformer):
        """A SourceFileLoader for notebooks that provides line number debugginer in the JSON source."""
        EXTENSION_SUFFIXES = '.ipynb',
        
        _compile = staticmethod(compile)
        _loads = staticmethod(loads)
        format = _transform = staticmethod(dedent)
                
        __slots__ = 'stdout', 'stderr', 'display', '_lazy', '_exceptions', 'globals'
        
        def __init__(
            self, fullname=None, path=None, *, 
            stdout=False, stderr=False, display=False,
            lazy=False, exceptions=ImportNbException, globals=None
        ): 
            SourceFileLoader.__init__(self, fullname, path)
            capture_output.__init__(self, stdout=stdout, stderr=stderr, display=display)
            self._lazy = lazy
            self._exceptions = exceptions
            self.globals = {} if globals is None else globals
        
        def __call__(self, fullname=None, path=None): 
            self= copy(self)
            return SourceFileLoader.__init__(self, str(fullname), str(path)) or self
        
        def visit(self, node): 
            node = super().visit(node)
            return ast.fix_missing_locations(super().visit(node))
        
        def create_module(self, spec):
            module = _new_module(spec.name)
            _init_module_attrs(spec, module)
            module.__exception__ = None
            module.__dict__.update(self.globals)
            return module
                                                
        def exec_module(self, module, **globals):
            """All exceptions specific in the context.
            """
            module.__dict__.update(globals)
            with capture_output(stdout=self.stdout, stderr=self.stderr, display=self.display) as out:
                module.__output__ = out
                try: 
                    super().exec_module(module)
                except self._exceptions as e:
                    """Display a message if an error is escaped."""
                    module.__exception__ = e
                    warn('.'.join([
                        """{name} was partially imported with a {error}""".format(
                            error = type(e), name=module.__name__
                        ), "="*10, format_exc()]))
                        
        def _data_to_ast(self, data):
            if isinstance(data, bytes):
                data = self._loads(data.decode('utf-8'))
            return ast.Module(body=sum((
                cell_to_ast(object, transform=self.format).body 
                for object in data['cells']), []))
        
        def source_to_code(self, data, path):
            return self._compile(
                self.visit(self._data_to_ast(data)), path or '<notebook-compiled>', 'exec')
        
        from_filename = from_resource

In [11]:
    if __name__ == '__main__':
        m = Notebook().from_filename('loader.ipynb')

### Partial Loader

In [12]:
    class Partial(Notebook): 
        """A partial import tool for notebooks.
        
        Sometimes notebooks don't work, but there may be useful code!
        
        with Partial():
            import Untitled as nb
            assert nb.__exception__
        
        if isinstance(nb.__exception__, AssertionError):
            print("There was a False assertion.")
                
        Partial is useful in logging specific debugging approaches to the exception.
        """
        __init__ = partialmethod(Notebook.__init__, exceptions=BaseException)

### Lazy Loader

The lazy loader is helpful for time consuming operations.  The module is not evaluated until it is used the first time after loading.

In [13]:
    class Lazy(Notebook): 
        """A lazy importer for notebooks.  For long operations and a lot of data, the lazy importer delays imports until 
        an attribute is accessed the first time.
        
        with Lazy():
            import Untitled as nb
        """
        __init__ = partialmethod(Notebook.__init__, lazy=True)

# IPython Extensions

In [14]:
    def load_ipython_extension(ip=None): 
        add_path_hooks(Notebook, Notebook.EXTENSION_SUFFIXES)
    def unload_ipython_extension(ip=None): 
        remove_one_path_hook(Notebook)

In [15]:
    def main(*files):
        with ExitStack() as stack:
            loader = stack.enter_context(Notebook('__main__'))
            if not files:
                files = sys.argv[1:]
            for file in files:
                loader.from_filename(file)

# Developer

In [16]:
    if __name__ ==  '__main__':
        try:  from utils.export import export
        except: from .utils.export import export
        export('loader.ipynb', '../loader.py')
        m = Notebook().from_filename('loader.ipynb')
        __import__('doctest').testmod(m, verbose=2)

Trying:
    m = Notebook().from_filename('loader.ipynb', 'importnb.notebooks')
Expecting nothing
ok
Trying:
    assert m and m.Notebook
Expecting nothing
ok
Trying:
    with Partial(): 
        from importnb.notebooks import loader
Expecting nothing
ok
Trying:
    assert loader.__exception__ is None
Expecting nothing
ok
Trying:
    with Lazy(): 
        from importnb.notebooks import loader
Expecting nothing
ok
Trying:
    from importnb.loader import from_resource
Expecting nothing
ok
Trying:
    assert from_resource(Notebook(), 'loader.ipynb', 'importnb.notebooks')
Expecting nothing
ok
Trying:
    assert Notebook().from_filename('loader.ipynb', 'importnb.notebooks')
Expecting nothing
ok
Trying:
    assert Notebook().from_filename(m.__file__)
Expecting nothing
ok
Trying:
    with Notebook(stdout=True, stderr=True, display=True, globals=dict(show=True)):
        from importnb.notebooks import loader
Expecting nothing
ok
Trying:
    assert loader.__output__
Expecting nothing
ok
Trying:
 

    if __name__ ==  '__main__':
        __import__('doctest').testmod(Notebook().from_filename('loader.ipynb'), verbose=2)